# Import the needed libraries

In [17]:
# Packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import ticker
import seaborn as sns
import joypy
import category_encoders as ce
import collections
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import InterclusterDistance
import random
import re
from collections import defaultdict
import time
import warnings
warnings.filterwarnings("ignore")

# Encoding
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

# Data Partition
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# Data Normalization
from sklearn.preprocessing import MinMaxScaler

# Wrapper Methods
from sklearn.feature_selection import RFE

# Logistic Regression
from sklearn.linear_model import LogisticRegression

# Other models from sklearn
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import SGDClassifier, Perceptron, RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

# Model selection with grid search
from sklearn.model_selection import GridSearchCV

# Extract the data

In [18]:
# extract the data of the first sheet of Case4_UNL-IMS - Data - delivery v2.xlsx
ds_task_execution = pd.read_excel('Case4_UNL-IMS - Data - delivery v2.xlsx')
ds_task_execution_original = ds_task_execution.copy()

In [19]:
# extract the data of the second sheet of Case4_UNL-IMS - Data - delivery v2.xlsx
ds_user_info = pd.read_excel('Case4_UNL-IMS - Data - delivery v2.xlsx', sheet_name=1)
ds_user_info_original = ds_user_info.copy()

In [20]:
# extract the data of the third sheet of Case4_UNL-IMS - Data - delivery v2.xlsx
ds_specific_request = pd.read_excel('Case4_UNL-IMS - Data - delivery v2.xlsx', sheet_name=2)
ds_specific_request_original = ds_specific_request.copy()

In [21]:
# extract the data of the fourth sheet of Case4_UNL-IMS - Data - delivery v2.xlsx
ds_rejections = pd.read_excel('Case4_UNL-IMS - Data - delivery v2.xlsx', sheet_name=3)
ds_rejections_original = ds_rejections.copy()

# 1. Data Exploration and Preparation

### 1.1. Task Execution 

In [22]:
ds_task_execution.head()

,Task Id,Request Identifier,Task arrival date,Task capture date,Task execution end date,Task predicted end date,Actvity ID,Task Executer,Task executer department,Task Type,Action,idBPMApplicationAction
0,1584303,1,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,NaN,100,7638.0,569.0,Initial Request,Submit initial request,270
1,1584304,1,2022-04-11 16:02:14.743,2022-04-12 09:29:40.683,2022-04-12 10:02:54.687,2022-04-12 14:02:14.803,102,4322.0,1104.0,Execution,NaN,273
2,1585980,1,2022-04-12 10:02:55.530,2022-04-12 10:14:39.290,2022-04-12 10:17:16.050,2022-04-12 17:02:55.577,103,10606.0,NaN,Execution,NaN,273
3,1586077,1,2022-04-12 10:17:16.427,2022-04-12 10:22:47.610,2022-05-10 13:43:32.203,2022-04-12 17:17:16.490,104,7638.0,569.0,Execution,Task executed with success,282
4,1671448,1,2022-05-10 13:43:32.627,2022-05-10 13:47:53.570,2022-05-10 13:48:44.353,2022-05-17 08:43:32.737,107,7638.0,569.0,Final task,Request accepted by requester,299


In [23]:
ds_task_execution.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209017 entries, 0 to 209016
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Task Id                   209017 non-null  int64  
 1   Request Identifier        209017 non-null  int64  
 2   Task arrival date         209017 non-null  object 
 3   Task capture date         209017 non-null  object 
 4   Task execution end date   209017 non-null  object 
 5   Task predicted end date   163232 non-null  object 
 6   Actvity ID                209017 non-null  int64  
 7   Task Executer             188888 non-null  float64
 8   Task executer department  181887 non-null  float64
 9   Task Type                 209017 non-null  object 
 10  Action                    124576 non-null  object 
 11  idBPMApplicationAction    209017 non-null  int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 19.1+ MB


In [24]:
# Rename the column Actvity ID to Activity ID
ds_task_execution.rename(columns={'Actvity ID': 'Activity ID'}, inplace=True)

In [25]:
# Check for nulls
ds_task_execution.isnull().sum()

Task Id                         0
Request Identifier              0
Task arrival date               0
Task capture date               0
Task execution end date         0
Task predicted end date     45785
Activity ID                     0
Task Executer               20129
Task executer department    27130
Task Type                       0
Action                      84441
idBPMApplicationAction          0
dtype: int64

In [26]:
# 'Task predicted end date' nulls are all of task type 'Initial Request'

In [27]:
# Check for duplicates
ds_task_execution.duplicated().sum()

0

In [28]:
# Check 'Task Id' values
ds_task_execution['Task Id'].value_counts()

1584303    1
3945953    1
3945749    1
3947939    1
3948764    1
          ..
2528343    1
2528344    1
2531841    1
2532500    1
6863412    1
Name: Task Id, Length: 209017, dtype: int64

In [29]:
# Check 'Request Identifier' values
ds_task_execution['Request Identifier'].value_counts()

38251    26
14835    22
42129    22
27606    21
38766    20
         ..
7698      2
39088     2
27537     2
29429     2
45449     2
Name: Request Identifier, Length: 45772, dtype: int64

In [30]:
# Create a new column with the last Activity ID for each Request Identifier
ds_task_execution['Last Activity ID'] = ds_task_execution.groupby('Request Identifier')['Activity ID'].transform('last')
ds_task_execution.head(12)

,Task Id,Request Identifier,Task arrival date,Task capture date,Task execution end date,Task predicted end date,Activity ID,Task Executer,Task executer department,Task Type,Action,idBPMApplicationAction,Last Activity ID
0,1584303,1,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,NaN,100,7638.0,569.0,Initial Request,Submit initial request,270,107
1,1584304,1,2022-04-11 16:02:14.743,2022-04-12 09:29:40.683,2022-04-12 10:02:54.687,2022-04-12 14:02:14.803,102,4322.0,1104.0,Execution,NaN,273,107
2,1585980,1,2022-04-12 10:02:55.530,2022-04-12 10:14:39.290,2022-04-12 10:17:16.050,2022-04-12 17:02:55.577,103,10606.0,NaN,Execution,NaN,273,107
3,1586077,1,2022-04-12 10:17:16.427,2022-04-12 10:22:47.610,2022-05-10 13:43:32.203,2022-04-12 17:17:16.490,104,7638.0,569.0,Execution,Task executed with success,282,107
4,1671448,1,2022-05-10 13:43:32.627,2022-05-10 13:47:53.570,2022-05-10 13:48:44.353,2022-05-17 08:43:32.737,107,7638.0,569.0,Final task,Request accepted by requester,299,107
5,1586253,2,2022-04-12 10:50:25.570,2022-04-12 10:50:25.570,2022-04-12 10:50:25.570,NaN,100,598.0,606.0,Initial Request,Submit initial request,270,107
6,1586254,2,2022-04-12 10:50:26.147,2022-04-12 10:52:00.203,2022-04-12 12:35:56.390,2022-04-13 08:50:26.197,102,4322.0,1104.0,Execution,NaN,273,107
7,1587111,2,2022-04-12 12:35:56.890,2022-04-12 13:44:25.607,2022-04-12 13:45:24.067,2022-04-13 10:35:56.930,103,10606.0,NaN,Execution,NaN,273,107
8,1587525,2,2022-04-12 13:45:24.847,2022-04-12 13:53:42.120,2022-04-12 14:18:35.850,2022-04-13 11:45:24.960,104,598.0,606.0,Execution,Task executed with success,282,107
9,1587689,2,2022-04-12 14:18:36.880,2022-04-12 14:21:47.677,2022-04-12 14:23:32.340,2022-04-20 09:18:36.940,107,598.0,606.0,Final task,Request accepted by requester,299,107


In [31]:
# Check all the last activities
ds_task_execution['Last Activity ID'].value_counts()

107    123202
104     79014
101      6485
102       306
105         6
103         4
Name: Last Activity ID, dtype: int64

In [32]:
# Check idBPMApplicationAction unique values
ds_task_execution['idBPMApplicationAction'].unique()

array([ 270,  273,  282,  299,  298,  290,  271, 2981, 8888,  888,  278,
        272,  300,  301,  274,  279,  275,  777, 2982], dtype=int64)

In [33]:
# Example dataframe when the 'Activity ID' is 107
ds_task_execution[ds_task_execution['Activity ID'] == 107]

,Task Id,Request Identifier,Task arrival date,Task capture date,Task execution end date,Task predicted end date,Activity ID,Task Executer,Task executer department,Task Type,Action,idBPMApplicationAction,Last Activity ID
4,1671448,1,2022-05-10 13:43:32.627,2022-05-10 13:47:53.570,2022-05-10 13:48:44.353,2022-05-17 08:43:32.737,107,7638.0,569.0,Final task,Request accepted by requester,299,107
9,1587689,2,2022-04-12 14:18:36.880,2022-04-12 14:21:47.677,2022-04-12 14:23:32.340,2022-04-20 09:18:36.940,107,598.0,606.0,Final task,Request accepted by requester,299,107
35,1839129,8,2022-06-14 16:13:12.947,2022-06-14 16:46:06.883,2022-06-14 16:46:46.810,2022-06-22 11:13:13.097,107,7638.0,569.0,Final task,Request accepted by requester,299,107
40,1593793,9,2022-04-14 09:46:55.640,2022-04-14 09:47:36.313,2022-04-14 09:47:54.830,2022-04-21 13:46:55.720,107,10847.0,606.0,Final task,Request accepted by requester,299,107
53,1602749,12,2022-04-19 16:02:08.233,2022-04-27 11:02:08.633,2022-04-27 11:02:08.633,2022-04-27 11:02:08.297,107,NaN,NaN,Final task,Task automaticaly terminated - SLA time reached,888,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208995,6692068,45768,2024-04-09 15:32:39.317,2024-04-16 10:32:40.020,2024-04-16 10:32:40.020,2024-04-16 10:32:39.390,107,NaN,NaN,Final task,Task automaticaly terminated - SLA time reached,888,107
209001,6638514,45769,2024-04-02 11:34:06.487,2024-04-08 15:34:06.770,2024-04-08 15:34:06.770,2024-04-08 15:34:06.540,107,NaN,NaN,Final task,Task automaticaly terminated - SLA time reached,888,107
209006,6631830,45770,2024-04-01 14:31:21.200,2024-04-01 14:31:42.607,2024-04-01 14:32:14.160,2024-04-08 09:31:21.233,107,7351.0,585.0,Final task,Request accepted by requester,299,107
209010,6729661,45771,2024-04-16 09:15:42.380,2024-04-16 09:19:19.753,2024-04-16 09:20:08.637,2024-04-22 13:15:42.443,107,7945.0,478.0,Final task,Request accepted by requester,299,107


In [34]:
# Define the target based on the last activity
last_activity_target_map = {
    107: 'Request Finished', 
    102: 'Request Canceled',
    101: 'Request Canceled',
    104: 'Closed administratively Requester Rejects Accounting Impact',
    106: 'Closed administratively', 
    103: 'Closed administratively',
    108: 'Closed administratively'
}

In [35]:
# Function to determine the Target based on the column 'Last Activity ID' 
def determine_target(row):
    if row['Last Activity ID'] == 107:
        if row['idBPMApplicationAction'] == 299:
            return 'Request Finished'
        elif row['idBPMApplicationAction'] == 888:
            return 'Closed administratively'
        else:
            return last_activity_target_map.get(row['Last Activity ID'], np.nan)
    else:
        return last_activity_target_map.get(row['Last Activity ID'], np.nan)

In [36]:
# Create new column with the output and drop the 'Last Activity ID' column 
ds_task_execution['Target'] = ds_task_execution.apply(determine_target, axis=1)

ds_task_execution = ds_task_execution.drop(columns=['Last Activity ID'])

ds_task_execution.head()

,Task Id,Request Identifier,Task arrival date,Task capture date,Task execution end date,Task predicted end date,Activity ID,Task Executer,Task executer department,Task Type,Action,idBPMApplicationAction,Target
0,1584303,1,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,NaN,100,7638.0,569.0,Initial Request,Submit initial request,270,Request Finished
1,1584304,1,2022-04-11 16:02:14.743,2022-04-12 09:29:40.683,2022-04-12 10:02:54.687,2022-04-12 14:02:14.803,102,4322.0,1104.0,Execution,NaN,273,Request Finished
2,1585980,1,2022-04-12 10:02:55.530,2022-04-12 10:14:39.290,2022-04-12 10:17:16.050,2022-04-12 17:02:55.577,103,10606.0,NaN,Execution,NaN,273,Request Finished
3,1586077,1,2022-04-12 10:17:16.427,2022-04-12 10:22:47.610,2022-05-10 13:43:32.203,2022-04-12 17:17:16.490,104,7638.0,569.0,Execution,Task executed with success,282,Request Finished
4,1671448,1,2022-05-10 13:43:32.627,2022-05-10 13:47:53.570,2022-05-10 13:48:44.353,2022-05-17 08:43:32.737,107,7638.0,569.0,Final task,Request accepted by requester,299,Request Finished


In [37]:
# Check the example dataframe with the new column
ds_task_execution[ds_task_execution['Activity ID'] == 107]

,Task Id,Request Identifier,Task arrival date,Task capture date,Task execution end date,Task predicted end date,Activity ID,Task Executer,Task executer department,Task Type,Action,idBPMApplicationAction,Target
4,1671448,1,2022-05-10 13:43:32.627,2022-05-10 13:47:53.570,2022-05-10 13:48:44.353,2022-05-17 08:43:32.737,107,7638.0,569.0,Final task,Request accepted by requester,299,Request Finished
9,1587689,2,2022-04-12 14:18:36.880,2022-04-12 14:21:47.677,2022-04-12 14:23:32.340,2022-04-20 09:18:36.940,107,598.0,606.0,Final task,Request accepted by requester,299,Request Finished
35,1839129,8,2022-06-14 16:13:12.947,2022-06-14 16:46:06.883,2022-06-14 16:46:46.810,2022-06-22 11:13:13.097,107,7638.0,569.0,Final task,Request accepted by requester,299,Request Finished
40,1593793,9,2022-04-14 09:46:55.640,2022-04-14 09:47:36.313,2022-04-14 09:47:54.830,2022-04-21 13:46:55.720,107,10847.0,606.0,Final task,Request accepted by requester,299,Request Finished
53,1602749,12,2022-04-19 16:02:08.233,2022-04-27 11:02:08.633,2022-04-27 11:02:08.633,2022-04-27 11:02:08.297,107,NaN,NaN,Final task,Task automaticaly terminated - SLA time reached,888,Closed administratively
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208995,6692068,45768,2024-04-09 15:32:39.317,2024-04-16 10:32:40.020,2024-04-16 10:32:40.020,2024-04-16 10:32:39.390,107,NaN,NaN,Final task,Task automaticaly terminated - SLA time reached,888,Closed administratively
209001,6638514,45769,2024-04-02 11:34:06.487,2024-04-08 15:34:06.770,2024-04-08 15:34:06.770,2024-04-08 15:34:06.540,107,NaN,NaN,Final task,Task automaticaly terminated - SLA time reached,888,Closed administratively
209006,6631830,45770,2024-04-01 14:31:21.200,2024-04-01 14:31:42.607,2024-04-01 14:32:14.160,2024-04-08 09:31:21.233,107,7351.0,585.0,Final task,Request accepted by requester,299,Request Finished
209010,6729661,45771,2024-04-16 09:15:42.380,2024-04-16 09:19:19.753,2024-04-16 09:20:08.637,2024-04-22 13:15:42.443,107,7945.0,478.0,Final task,Request accepted by requester,299,Request Finished


### 1.2. User Information

In [38]:
ds_user_info.head()

,Task Executer,Sex,BirthYear,Role ID,Is Manager,OrgUnitSince,IsOutSourcer
0,1872,F,1988,1,Yes,2024,N
1,11133,F,1975,1,Yes,2019,N
2,10332,F,1978,2,Yes,2019,N
3,10349,F,1976,3,Yes,2024,N
4,1394,F,1984,4,Yes,2021,N


In [39]:
ds_user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11370 entries, 0 to 11369
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Task Executer  11370 non-null  int64 
 1   Sex            11370 non-null  object
 2   BirthYear      11370 non-null  int64 
 3   Role ID        11370 non-null  int64 
 4   Is Manager     11370 non-null  object
 5   OrgUnitSince   11370 non-null  int64 
 6   IsOutSourcer   11370 non-null  object
dtypes: int64(4), object(3)
memory usage: 621.9+ KB


In [40]:
# Check for nulls
ds_user_info.isnull().sum()

Task Executer    0
Sex              0
BirthYear        0
Role ID          0
Is Manager       0
OrgUnitSince     0
IsOutSourcer     0
dtype: int64

In [41]:
# Check for duplicates
ds_user_info.duplicated().sum()

24

In [42]:
# Remove duplicates
ds_user_info.drop_duplicates(inplace=True)
ds_user_info.duplicated().sum()

0

In [43]:
# Check 'Task Executer' values
ds_user_info['Task Executer'].value_counts()

7779     6
7457     5
6955     5
1341     5
1728     5
        ..
10606    1
8650     1
8172     1
8557     1
1257     1
Name: Task Executer, Length: 11274, dtype: int64

In [44]:
# Check how many Task Executers have multiple data info
y = ds_user_info['Task Executer'].value_counts()
count_2_or_more = y[y >= 2].count()
print(count_2_or_more)

45


In [45]:
# Rows with task executer 1728 for example
task_executer_1728 = ds_user_info[ds_user_info['Task Executer'] == 1728]
print(task_executer_1728)

       Task Executer Sex  BirthYear  Role ID Is Manager  OrgUnitSince  \
1390            1728   M       1963       74        Yes          2022   
5420            1728   M       1963       80        Yes          2018   
7106            1728   M       1963       73        Yes          2022   
9970            1728   M       1963       75        Yes          2022   
10020           1728   M       1963       71        Yes          2018   

      IsOutSourcer  
1390             N  
5420             N  
7106             N  
9970             N  
10020            N  


In [46]:
# Check if the repeated task executers are in the dataframe ds_task_execution

repeated_task_executers = ds_user_info['Task Executer'].value_counts()
repeated_task_executers = repeated_task_executers[repeated_task_executers > 1].index

task_executers_in_ds_task_execution = set(ds_task_execution['Task Executer'].unique())

if all(task_executer not in task_executers_in_ds_task_execution for task_executer in repeated_task_executers):
    print("All repeated task executers in ds_user_info are not in ds_task_execution.")
else:
    print("There are repeated task executers in ds_user_info that are also in ds_task_execution.")

All repeated task executers in ds_user_info are not in ds_task_execution.


In [47]:
# Drop repeated 'Task Executer' values
ds_user_info = ds_user_info[~ds_user_info['Task Executer'].isin(repeated_task_executers)]

In [48]:
# Check 'Task Executer' values
ds_user_info['Task Executer'].value_counts()

1872     1
11210    1
4302     1
10781    1
8198     1
        ..
4309     1
6961     1
7185     1
932      1
1257     1
Name: Task Executer, Length: 11229, dtype: int64

### 1.3. Specific Request

In [49]:
ds_specific_request.head()

,idField,Request Identifier,Value
0,47,328,0
1,47,327,0
2,47,291,0
3,47,289,0
4,47,219,0


In [50]:
ds_specific_request.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297556 entries, 0 to 297555
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   idField             297556 non-null  int64 
 1   Request Identifier  297556 non-null  int64 
 2   Value               297556 non-null  object
dtypes: int64(2), object(1)
memory usage: 6.8+ MB


In [51]:
# Check for nulls
ds_specific_request.isnull().sum()

idField               0
Request Identifier    0
Value                 0
dtype: int64

In [52]:
# Check for duplicates
ds_specific_request.duplicated().sum()

0

In [53]:
# Check 'Request Identifier' values
ds_specific_request['Request Identifier'].value_counts()

328      10
4282     10
3525     10
3480     10
3445     10
         ..
20478     4
20376     4
22208     4
21248     4
37102     4
Name: Request Identifier, Length: 45772, dtype: int64

In [54]:
# Rows with Request Identifier 328 for example
request_identifier_328 = ds_specific_request[ds_specific_request['Request Identifier'] == 328]
print(request_identifier_328)

        idField  Request Identifier                     Value
0            47                 328                         0
16         1602                 328                         0
88978      3491                 328                        10
89147      3420                 328                        10
89476      3486                 328                        10
89856      3388                 328                        10
90166      3390                 328                        10
90544      3391                 328                        10
139116      203                 328  2022-05-08T23:00:00.000Z
290060     1604                 328                        80


In [55]:
# Function to replace strange values in the column 'Value' with Unknown
def replace_values(value):
    if isinstance(value, str) and re.match(r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d{3}Z', value):
        return 'Unknown'
    return value

In [56]:
# Apply the function to replace values
ds_specific_request['Value'] = ds_specific_request['Value'].apply(replace_values)

In [57]:
# Check new values
ds_specific_request['Value'].unique()

array([0, 1, -1, 10, 100, 2, -2, 20, 'Unknown', 30, 40, 5, 50, 6, 60, 70,
       80, 90], dtype=object)

In [58]:
# Check Request Identifier 328 example
request_identifier_328 = ds_specific_request[ds_specific_request['Request Identifier'] == 328]
print(request_identifier_328)

        idField  Request Identifier    Value
0            47                 328        0
16         1602                 328        0
88978      3491                 328       10
89147      3420                 328       10
89476      3486                 328       10
89856      3388                 328       10
90166      3390                 328       10
90544      3391                 328       10
139116      203                 328  Unknown
290060     1604                 328       80


In [59]:
# Check 'Value' values cardinality
ds_specific_request['Value'].value_counts()

10         85761
Unknown    45772
50         35153
30         28320
70         22450
20         20092
100        19520
60         12935
2           7988
40          6075
1           4681
80          4081
90          3579
5            353
0            246
-1           209
-2           178
6            163
Name: Value, dtype: int64

### 1.4. Rejections

In [60]:
ds_rejections.head()

,Task Id,idBPMRequirement
0,1606251,191
1,1606786,183
2,1629944,183
3,1643257,183
4,1643929,183


In [61]:
ds_rejections.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4099 entries, 0 to 4098
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Task Id           4099 non-null   int64
 1   idBPMRequirement  4099 non-null   int64
dtypes: int64(2)
memory usage: 64.2 KB


In [62]:
# Check for nulls
ds_rejections.isnull().sum()

Task Id             0
idBPMRequirement    0
dtype: int64

In [63]:
# Check for duplicates
ds_rejections.duplicated().sum()

0

In [64]:
# Check 'Task Id' values cardinality
ds_rejections['Task Id'].value_counts()

1654061    7
1647860    6
1925405    2
2999956    2
3139691    2
          ..
3266189    1
3266202    1
3266928    1
3267685    1
6636734    1
Name: Task Id, Length: 4056, dtype: int64

In [65]:
# Rows with Task Id 1654061 for example
task_id_1654061 = ds_rejections[ds_rejections['Task Id'] == 1654061]
print(task_id_1654061)

    Task Id  idBPMRequirement
20  1654061               183
21  1654061               191
22  1654061               300
23  1654061               331
24  1654061               353
25  1654061               661
26  1654061              1436


### 1.5. Merge the datasets

In [66]:
# Merge the Task Execution with the User Information by Task Executer
ds_joined = pd.merge(ds_task_execution, ds_user_info, on='Task Executer', how='left')
ds_joined.head()

,Task Id,Request Identifier,Task arrival date,Task capture date,Task execution end date,Task predicted end date,Activity ID,Task Executer,Task executer department,Task Type,Action,idBPMApplicationAction,Target,Sex,BirthYear,Role ID,Is Manager,OrgUnitSince,IsOutSourcer
0,1584303,1,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,NaN,100,7638.0,569.0,Initial Request,Submit initial request,270,Request Finished,M,1967.0,5.0,Yes,2021.0,N
1,1584304,1,2022-04-11 16:02:14.743,2022-04-12 09:29:40.683,2022-04-12 10:02:54.687,2022-04-12 14:02:14.803,102,4322.0,1104.0,Execution,NaN,273,Request Finished,F,1967.0,21.0,No,2024.0,Y
2,1585980,1,2022-04-12 10:02:55.530,2022-04-12 10:14:39.290,2022-04-12 10:17:16.050,2022-04-12 17:02:55.577,103,10606.0,NaN,Execution,NaN,273,Request Finished,F,1975.0,11.0,Yes,2023.0,N
3,1586077,1,2022-04-12 10:17:16.427,2022-04-12 10:22:47.610,2022-05-10 13:43:32.203,2022-04-12 17:17:16.490,104,7638.0,569.0,Execution,Task executed with success,282,Request Finished,M,1967.0,5.0,Yes,2021.0,N
4,1671448,1,2022-05-10 13:43:32.627,2022-05-10 13:47:53.570,2022-05-10 13:48:44.353,2022-05-17 08:43:32.737,107,7638.0,569.0,Final task,Request accepted by requester,299,Request Finished,M,1967.0,5.0,Yes,2021.0,N


In [67]:
ds_joined.shape

(209017, 19)

In [68]:
# Aggregate the columns 'idField' and 'Value' from ds_specific_request to merge with ds_joined
ds_specific_request_aggregated = ds_specific_request.groupby('Request Identifier').agg({
    'idField': lambda x: ','.join(map(str, x)),
    'Value': lambda x: ','.join(map(str, x))
}).reset_index()

ds_specific_request_aggregated.head()

,Request Identifier,idField,Value
0,1,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90"
1,2,"3486,3491,3388,3390,3391,3420,203","10,10,10,10,10,20,Unknown"
2,3,"3390,3420,203,3388,3486","10,10,Unknown,30,30"
3,4,"3486,3388,3390,3420,203","10,10,10,20,Unknown"
4,5,"3391,3420,3491,3388,203,3390,3486","10,10,20,20,Unknown,30,30"


In [69]:
# Merge the ds_joined with the ds_specific_request_aggregated by Request Identifier
ds_joined = pd.merge(ds_joined, ds_specific_request_aggregated, on='Request Identifier', how='left')
ds_joined.head()

,Task Id,Request Identifier,Task arrival date,Task capture date,Task execution end date,Task predicted end date,Activity ID,Task Executer,Task executer department,Task Type,...,idBPMApplicationAction,Target,Sex,BirthYear,Role ID,Is Manager,OrgUnitSince,IsOutSourcer,idField,Value
0,1584303,1,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,NaN,100,7638.0,569.0,Initial Request,...,270,Request Finished,M,1967.0,5.0,Yes,2021.0,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90"
1,1584304,1,2022-04-11 16:02:14.743,2022-04-12 09:29:40.683,2022-04-12 10:02:54.687,2022-04-12 14:02:14.803,102,4322.0,1104.0,Execution,...,273,Request Finished,F,1967.0,21.0,No,2024.0,Y,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90"
2,1585980,1,2022-04-12 10:02:55.530,2022-04-12 10:14:39.290,2022-04-12 10:17:16.050,2022-04-12 17:02:55.577,103,10606.0,NaN,Execution,...,273,Request Finished,F,1975.0,11.0,Yes,2023.0,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90"
3,1586077,1,2022-04-12 10:17:16.427,2022-04-12 10:22:47.610,2022-05-10 13:43:32.203,2022-04-12 17:17:16.490,104,7638.0,569.0,Execution,...,282,Request Finished,M,1967.0,5.0,Yes,2021.0,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90"
4,1671448,1,2022-05-10 13:43:32.627,2022-05-10 13:47:53.570,2022-05-10 13:48:44.353,2022-05-17 08:43:32.737,107,7638.0,569.0,Final task,...,299,Request Finished,M,1967.0,5.0,Yes,2021.0,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90"


In [70]:
ds_joined.shape

(209017, 21)

In [71]:
# Aggregate the column 'idBPMRequirement' from ds_rejections to merge with ds_joined
ds_rejections_aggregated = ds_rejections.groupby('Task Id').agg({
    'idBPMRequirement': lambda x: ','.join(map(str, x))}).reset_index()

ds_rejections_aggregated.head()

,Task Id,idBPMRequirement
0,1606251,191
1,1606786,183
2,1629944,183
3,1643257,183
4,1643929,183


In [72]:
# Check if the values from the column 'idBPMRequirement' are aggregated
agg_task_id_1654061 = ds_rejections_aggregated[ds_rejections_aggregated['Task Id'] == 1654061]
print(agg_task_id_1654061)

    Task Id              idBPMRequirement
15  1654061  183,191,300,331,353,661,1436


In [73]:
# Merge the ds_joined with the ds_rejections_aggregated by Task Id
ds_joined = pd.merge(ds_joined, ds_rejections_aggregated, on='Task Id', how='left')
ds_joined.head()

,Task Id,Request Identifier,Task arrival date,Task capture date,Task execution end date,Task predicted end date,Activity ID,Task Executer,Task executer department,Task Type,...,Target,Sex,BirthYear,Role ID,Is Manager,OrgUnitSince,IsOutSourcer,idField,Value,idBPMRequirement
0,1584303,1,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,NaN,100,7638.0,569.0,Initial Request,...,Request Finished,M,1967.0,5.0,Yes,2021.0,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",NaN
1,1584304,1,2022-04-11 16:02:14.743,2022-04-12 09:29:40.683,2022-04-12 10:02:54.687,2022-04-12 14:02:14.803,102,4322.0,1104.0,Execution,...,Request Finished,F,1967.0,21.0,No,2024.0,Y,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",NaN
2,1585980,1,2022-04-12 10:02:55.530,2022-04-12 10:14:39.290,2022-04-12 10:17:16.050,2022-04-12 17:02:55.577,103,10606.0,NaN,Execution,...,Request Finished,F,1975.0,11.0,Yes,2023.0,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",NaN
3,1586077,1,2022-04-12 10:17:16.427,2022-04-12 10:22:47.610,2022-05-10 13:43:32.203,2022-04-12 17:17:16.490,104,7638.0,569.0,Execution,...,Request Finished,M,1967.0,5.0,Yes,2021.0,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",NaN
4,1671448,1,2022-05-10 13:43:32.627,2022-05-10 13:47:53.570,2022-05-10 13:48:44.353,2022-05-17 08:43:32.737,107,7638.0,569.0,Final task,...,Request Finished,M,1967.0,5.0,Yes,2021.0,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",NaN


In [74]:
ds_joined.shape

(209017, 22)

In [75]:
# Clean data values
columns_to_convert = ['Task Executer', 'Task executer department', 'BirthYear', 'Role ID', 'OrgUnitSince']

for col in columns_to_convert:
    ds_joined[col] = ds_joined[col].astype(str).str.rstrip('.0')
    ds_joined[col] = ds_joined[col].replace('nan', np.nan)

In [76]:
ds_joined.head()

,Task Id,Request Identifier,Task arrival date,Task capture date,Task execution end date,Task predicted end date,Activity ID,Task Executer,Task executer department,Task Type,...,Target,Sex,BirthYear,Role ID,Is Manager,OrgUnitSince,IsOutSourcer,idField,Value,idBPMRequirement
0,1584303,1,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,NaN,100,7638,569,Initial Request,...,Request Finished,M,1967,5,Yes,2021,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",NaN
1,1584304,1,2022-04-11 16:02:14.743,2022-04-12 09:29:40.683,2022-04-12 10:02:54.687,2022-04-12 14:02:14.803,102,4322,1104,Execution,...,Request Finished,F,1967,21,No,2024,Y,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",NaN
2,1585980,1,2022-04-12 10:02:55.530,2022-04-12 10:14:39.290,2022-04-12 10:17:16.050,2022-04-12 17:02:55.577,103,10606,NaN,Execution,...,Request Finished,F,1975,11,Yes,2023,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",NaN
3,1586077,1,2022-04-12 10:17:16.427,2022-04-12 10:22:47.610,2022-05-10 13:43:32.203,2022-04-12 17:17:16.490,104,7638,569,Execution,...,Request Finished,M,1967,5,Yes,2021,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",NaN
4,1671448,1,2022-05-10 13:43:32.627,2022-05-10 13:47:53.570,2022-05-10 13:48:44.353,2022-05-17 08:43:32.737,107,7638,569,Final task,...,Request Finished,M,1967,5,Yes,2021,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",NaN


In [77]:
# Change the columns data types
cols_to_object = ['Task Id', 'Request Identifier', 'Activity ID', 'Task Executer', 'Task executer department', 'idBPMApplicationAction', 'Role ID']
for col in cols_to_object:
    ds_joined[col] = ds_joined[col].astype('object')

ds_joined['Task arrival date'] = pd.to_datetime(ds_joined['Task arrival date'])
ds_joined['Task capture date'] = pd.to_datetime(ds_joined['Task capture date'])
ds_joined['Task execution end date'] = pd.to_datetime(ds_joined['Task execution end date'])
ds_joined['Task predicted end date'] = pd.to_datetime(ds_joined['Task predicted end date'])

In [78]:
ds_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209017 entries, 0 to 209016
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   Task Id                   209017 non-null  object        
 1   Request Identifier        209017 non-null  object        
 2   Task arrival date         209017 non-null  datetime64[ns]
 3   Task capture date         209017 non-null  datetime64[ns]
 4   Task execution end date   209017 non-null  datetime64[ns]
 5   Task predicted end date   163232 non-null  datetime64[ns]
 6   Activity ID               209017 non-null  object        
 7   Task Executer             188888 non-null  object        
 8   Task executer department  181887 non-null  object        
 9   Task Type                 209017 non-null  object        
 10  Action                    124576 non-null  object        
 11  idBPMApplicationAction    209017 non-null  object        
 12  Ta

### 1.6. Outliers

In [79]:
# Looping cases and possible outliers
ds_joined['Request Identifier'].value_counts()

38251    26
14835    22
42129    22
27606    21
38766    20
         ..
7698      2
39088     2
27537     2
29429     2
45449     2
Name: Request Identifier, Length: 45772, dtype: int64

In [80]:
# Mean of 'Request Identifier' values cardinality
ds_joined['Request Identifier'].value_counts().mean()

4.566481691864022

In [81]:
# Check possible looping cases
x = ds_joined['Request Identifier'].value_counts()
count_8_or_more = x[x >= 8].count()
print(count_8_or_more)

1821


In [82]:
# Drop looping cases
request_identifier_counts = ds_joined['Request Identifier'].value_counts()
looping_cases = request_identifier_counts[request_identifier_counts >= 8].index

ds_joined = ds_joined[~ds_joined['Request Identifier'].isin(looping_cases)]

In [83]:
ds_joined.shape

(192019, 22)

### 1.7. Missing Values
     
     We will take different approaches to impute missing values of different variables, depending on the % of missing values and what makes more sense given the context and our understanding of the variable in cause 

In [84]:
ds_joined.isna().sum()

Task Id                          0
Request Identifier               0
Task arrival date                0
Task capture date                0
Task execution end date          0
Task predicted end date      43962
Activity ID                      0
Task Executer                18810
Task executer department     25982
Task Type                        0
Action                       76110
idBPMApplicationAction           0
Target                           6
Sex                          18810
BirthYear                    18810
Role ID                      18810
Is Manager                   18810
OrgUnitSince                 18810
IsOutSourcer                 18810
idField                          0
Value                            0
idBPMRequirement            189333
dtype: int64

#### - Task predicted end date

In [85]:
# Check how many times the 'Task type' is 'Initial Request'
task_type_initial_request = ds_joined[ds_joined['Task Type'] == 'Initial Request']
task_type_initial_request.shape

(43951, 22)

In [86]:
# Fill missing values with 'Not necessary' 
ds_joined['Task predicted end date'] = ds_joined['Task predicted end date'].fillna('Not necessary')

#### - Task Executer

In [87]:
ds_joined['Task Executer'].value_counts()

10606    30298
496      15591
4731     11809
4322     11068
10925     2186
         ...  
8            1
1683         1
1197         1
1959         1
10318        1
Name: Task Executer, Length: 1561, dtype: int64

In [88]:
# Fill missing values with Unknown
ds_joined['Task Executer'] = ds_joined['Task Executer'].fillna('Unknown')

#### - Task executer department

In [89]:
ds_joined['Task executer department'].value_counts()

1146    23384
1104    21184
1147    15769
1172     7182
1177     4233
        ...  
454         1
101         1
307         1
37          1
113         1
Name: Task executer department, Length: 377, dtype: int64

In [90]:
# Fill missing values with Unknown
ds_joined['Task executer department'] = ds_joined['Task executer department'].fillna('Unknown')

#### - Action

In [91]:
ds_joined['Action'].value_counts()

Submit initial request                             43954
Task executed with success                         22756
Task terminated - administrative closure           19364
Task automaticaly terminated - SLA time reached    15693
Request accepted by requester                       8491
Task returned to team                               5651
Name: Action, dtype: int64

In [92]:
# Fill missing values with Unknown
ds_joined['Action'] = ds_joined['Action'].fillna('Unknown')

#### - Output

In [93]:
ds_joined['Target'].value_counts()

Request Finished                                               95279
Closed administratively Requester Rejects Accounting Impact    76120
Closed administratively                                        14224
Request Canceled                                                6390
Name: Target, dtype: int64

In [94]:
# Check rows when the 'Target' is nan
ds_joined_output_nan = ds_joined[ds_joined['Target'].isna()]
ds_joined_output_nan

,Task Id,Request Identifier,Task arrival date,Task capture date,Task execution end date,Task predicted end date,Activity ID,Task Executer,Task executer department,Task Type,...,Target,Sex,BirthYear,Role ID,Is Manager,OrgUnitSince,IsOutSourcer,idField,Value,idBPMRequirement
39901,2183562,8934,2022-09-19 09:10:06.913,2022-09-19 09:10:06.913,2022-09-19 09:10:06.913,Not necessary,100,1147,175,Initial Request,...,NaN,F,1982,5,Yes,2023,N,"3388,203,3420,3486,3390","10,Unknown,30,50,90",NaN
39902,2183563,8934,2022-09-19 09:10:08.320,2022-09-19 09:23:05.917,2022-09-19 09:23:05.917,2022-09-19 16:10:08.340000,105,1147,175,Execution,...,NaN,F,1982,5,Yes,2023,N,"3388,203,3420,3486,3390","10,Unknown,30,50,90",NaN
102301,3157900,22787,2023-03-29 15:00:37.053,2023-03-29 15:00:37.053,2023-03-29 15:00:37.053,Not necessary,100,8303,707,Initial Request,...,NaN,M,197,6,Yes,2018,N,"3486,203,3388,3390","10,Unknown,30,90",NaN
102302,3157901,22787,2023-03-29 15:00:41.083,2023-03-29 15:06:46.070,2023-03-29 15:06:46.070,2023-03-30 13:00:41.100000,105,8303,707,Execution,...,NaN,M,197,6,Yes,2018,N,"3486,203,3388,3390","10,Unknown,30,90",NaN
171732,4449549,38736,2023-11-03 16:45:08.440,2023-11-03 16:45:08.440,2023-11-03 16:45:08.440,Not necessary,100,2109,157,Initial Request,...,NaN,F,1997,4,Yes,2024,N,"3486,203,3388,3390","10,Unknown,50,90",NaN
171733,4449550,38736,2023-11-03 16:45:08.747,2023-11-03 16:47:15.117,2023-11-03 16:47:15.117,2023-11-06 14:45:08.823000,105,2109,157,Execution,...,NaN,F,1997,4,Yes,2024,N,"3486,203,3388,3390","10,Unknown,50,90",NaN


In [95]:
# The 'Target' is nan when the last activity of the case is 105, but according to the schema, the process can´t end at 105

In [96]:
# Drop rows when 'Target' is nan
ds_joined = ds_joined.dropna(subset=['Target'])

In [97]:
# Check 'Target' values cardinality
ds_joined['Target'].value_counts()

Request Finished                                               95279
Closed administratively Requester Rejects Accounting Impact    76120
Closed administratively                                        14224
Request Canceled                                                6390
Name: Target, dtype: int64

In [98]:
ds_joined.shape

(192013, 22)

#### - User information data (Sex, BirthYear, Role ID, Is Manager, OrgUnitSince, IsOutSourcer)

In [99]:
# Check if all task executers from ds_task_execution are in ds_user_info
all_task_executers_in_ds_user_info = set(ds_user_info['Task Executer'].unique())
task_executers_in_ds_task_execution = set(ds_task_execution['Task Executer'].unique())

if all(task_executer in all_task_executers_in_ds_user_info for task_executer in task_executers_in_ds_task_execution):
    print("All task executers from ds_task_execution are in ds_user_info.")
else:
    print("There are task executers in ds_task_execution that are not in ds_user_info.")

There are task executers in ds_task_execution that are not in ds_user_info.


In [100]:
# Fill missing values with 'Unknown'

In [101]:
ds_joined['Sex'] = ds_joined['Sex'].fillna('Unknown')
ds_joined['BirthYear'] = ds_joined['BirthYear'].fillna('Unknown')
ds_joined['Role ID'] = ds_joined['Role ID'].fillna('Unknown')
ds_joined['Is Manager'] = ds_joined['Is Manager'].fillna('Unknown')
ds_joined['OrgUnitSince'] = ds_joined['OrgUnitSince'].fillna('Unknown')
ds_joined['IsOutSourcer'] = ds_joined['IsOutSourcer'].fillna('Unknown')

#### - idBPMRequirement

In [102]:
ds_joined['idBPMRequirement'].value_counts()

183                             1152
300                             1096
331                              155
191                              127
353                               79
661                               48
183,353                            8
183,300                            8
1436                               7
300,353                            2
183,191,300,331,353,661,1436       1
300,331                            1
183,191                            1
300,1436                           1
Name: idBPMRequirement, dtype: int64

In [103]:
# Fill missing values with 'Not Applicable'
ds_joined['idBPMRequirement'] = ds_joined['idBPMRequirement'].fillna('Not Applicable')

In [104]:
# Check if there are still missing values
ds_joined.isna().sum()

Task Id                     0
Request Identifier          0
Task arrival date           0
Task capture date           0
Task execution end date     0
Task predicted end date     0
Activity ID                 0
Task Executer               0
Task executer department    0
Task Type                   0
Action                      0
idBPMApplicationAction      0
Target                      0
Sex                         0
BirthYear                   0
Role ID                     0
Is Manager                  0
OrgUnitSince                0
IsOutSourcer                0
idField                     0
Value                       0
idBPMRequirement            0
dtype: int64

# 2. Preprocessing 

In [105]:
# Check dataframe
ds_joined.head(10)

,Task Id,Request Identifier,Task arrival date,Task capture date,Task execution end date,Task predicted end date,Activity ID,Task Executer,Task executer department,Task Type,...,Target,Sex,BirthYear,Role ID,Is Manager,OrgUnitSince,IsOutSourcer,idField,Value,idBPMRequirement
0,1584303,1,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,2022-04-11 16:02:13.820,Not necessary,100,7638,569,Initial Request,...,Request Finished,M,1967,5,Yes,2021,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",Not Applicable
1,1584304,1,2022-04-11 16:02:14.743,2022-04-12 09:29:40.683,2022-04-12 10:02:54.687,2022-04-12 14:02:14.803000,102,4322,1104,Execution,...,Request Finished,F,1967,21,No,2024,Y,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",Not Applicable
2,1585980,1,2022-04-12 10:02:55.530,2022-04-12 10:14:39.290,2022-04-12 10:17:16.050,2022-04-12 17:02:55.577000,103,10606,Unknown,Execution,...,Request Finished,F,1975,11,Yes,2023,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",Not Applicable
3,1586077,1,2022-04-12 10:17:16.427,2022-04-12 10:22:47.610,2022-05-10 13:43:32.203,2022-04-12 17:17:16.490000,104,7638,569,Execution,...,Request Finished,M,1967,5,Yes,2021,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",Not Applicable
4,1671448,1,2022-05-10 13:43:32.627,2022-05-10 13:47:53.570,2022-05-10 13:48:44.353,2022-05-17 08:43:32.737000,107,7638,569,Final task,...,Request Finished,M,1967,5,Yes,2021,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",Not Applicable
5,1586253,2,2022-04-12 10:50:25.570,2022-04-12 10:50:25.570,2022-04-12 10:50:25.570,Not necessary,100,598,606,Initial Request,...,Request Finished,F,1979,5,Yes,2023,N,"3486,3491,3388,3390,3391,3420,203","10,10,10,10,10,20,Unknown",Not Applicable
6,1586254,2,2022-04-12 10:50:26.147,2022-04-12 10:52:00.203,2022-04-12 12:35:56.390,2022-04-13 08:50:26.197000,102,4322,1104,Execution,...,Request Finished,F,1967,21,No,2024,Y,"3486,3491,3388,3390,3391,3420,203","10,10,10,10,10,20,Unknown",Not Applicable
7,1587111,2,2022-04-12 12:35:56.890,2022-04-12 13:44:25.607,2022-04-12 13:45:24.067,2022-04-13 10:35:56.930000,103,10606,Unknown,Execution,...,Request Finished,F,1975,11,Yes,2023,N,"3486,3491,3388,3390,3391,3420,203","10,10,10,10,10,20,Unknown",Not Applicable
8,1587525,2,2022-04-12 13:45:24.847,2022-04-12 13:53:42.120,2022-04-12 14:18:35.850,2022-04-13 11:45:24.960000,104,598,606,Execution,...,Request Finished,F,1979,5,Yes,2023,N,"3486,3491,3388,3390,3391,3420,203","10,10,10,10,10,20,Unknown",Not Applicable
9,1587689,2,2022-04-12 14:18:36.880,2022-04-12 14:21:47.677,2022-04-12 14:23:32.340,2022-04-20 09:18:36.940000,107,598,606,Final task,...,Request Finished,F,1979,5,Yes,2023,N,"3486,3491,3388,3390,3391,3420,203","10,10,10,10,10,20,Unknown",Not Applicable


In [106]:
ds_joined['OrgUnitSince'].unique()

array(['2021', '2024', '2023', '202', '2014', '2017', 'Unknown', '2022',
       '2019', '2016', '2013', '2009', '2015', '2018', '2011', '2012',
       '201', '2004', '2008', '2005', '2006', '2007', '2001'],
      dtype=object)

In [107]:
# Correct 'OrgUnitSince' column values
ds_joined['OrgUnitSince'] = ds_joined['OrgUnitSince'].replace({'197': '1970', '202': '2020'})

In [108]:
ds_joined['OrgUnitSince'].value_counts()

2023       71168
2024       43697
2022       21217
Unknown    18810
2021       11214
2020        6908
2019        6749
2018        2923
2016        2716
2017        2652
2012         746
2013         743
2014         713
2015         688
2009         418
201          240
2011         137
2004         127
2005          91
2008          26
2001          17
2007           7
2006           6
Name: OrgUnitSince, dtype: int64

In [109]:
# Correct 'BirthYear' column values
ds_joined['BirthYear'] = ds_joined['BirthYear'].replace({'201': '2010', '2': '2000', '196': '1960', '198': '1980', '197': '1970', '199': '1990'})

In [110]:
ds_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192013 entries, 0 to 209016
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   Task Id                   192013 non-null  object        
 1   Request Identifier        192013 non-null  object        
 2   Task arrival date         192013 non-null  datetime64[ns]
 3   Task capture date         192013 non-null  datetime64[ns]
 4   Task execution end date   192013 non-null  datetime64[ns]
 5   Task predicted end date   192013 non-null  object        
 6   Activity ID               192013 non-null  object        
 7   Task Executer             192013 non-null  object        
 8   Task executer department  192013 non-null  object        
 9   Task Type                 192013 non-null  object        
 10  Action                    192013 non-null  object        
 11  idBPMApplicationAction    192013 non-null  object        
 12  Ta

In [111]:
# Group 'BirthYear' values in decades

intervals = [
    (1950, 1959, "1950-1959"),
    (1960, 1969, "1960-1969"),
    (1970, 1979, "1970-1979"),
    (1980, 1989, "1980-1989"),
    (1990, 1999, "1990-1999"),
    (2000, 2009, "2000-2009")
]

# Function to group values 
def categorize_birthyear(year):
    if year == "Unknown":
        return "Unknown"
    year = int(year)
    for start, end, label in intervals:
        if start <= year <= end:
            return label
    return "Other"

# Apply the function 
ds_joined['BirthYear'] = ds_joined['BirthYear'].apply(categorize_birthyear)

In [112]:
# Group less common values into 'other' for different variables

# 'idBPMApplicationAction'
value_counts = ds_joined['idBPMApplicationAction'].value_counts()
mask = value_counts < 10000
ds_joined['idBPMApplicationAction'] = ds_joined['idBPMApplicationAction'].apply(
    lambda x: 'other' if mask[x] else str(x)
)

# 'Role ID'
value_counts = ds_joined['Role ID'].value_counts()
mask = value_counts < 5000
ds_joined['Role ID'] = ds_joined['Role ID'].apply(
    lambda x: 'other' if mask[x] else str(x)
)

# 'OrgUnitSince'
value_counts = ds_joined['OrgUnitSince'].value_counts()
mask = value_counts < 5000
ds_joined['OrgUnitSince'] = ds_joined['OrgUnitSince'].apply(
    lambda x: 'other' if mask[x] else str(x)
)

In [113]:
#Create a new variable 'Task predicted end date'

# Replace 'Not necessary' with nans and change the datatype to apply the function
ds_joined['Task predicted end date'] = ds_joined['Task predicted end date'].replace('Not necessary', np.nan)
ds_joined['Task predicted end date'] = pd.to_datetime(ds_joined['Task predicted end date'], errors='coerce')

# Function to create a new variable 'Task predicted end date' based on the columns 'Task execution end date' and 'Task predicted end date'  
def create_label(row):
    if pd.isna(row['Task predicted end date']):
        return 'Unknown'
    elif row['Task execution end date'] < row['Task predicted end date']:
        return 'Yes'
    else:
        return 'No'

# Apply the function
ds_joined['task_ended_before_predicted'] = ds_joined.apply(create_label, axis=1)

In [114]:
# Drop columns
ds_joined = ds_joined.drop(columns=['Task Id', 'Task arrival date', 'Task capture date', 'Task execution end date', 'Task predicted end date'])

In [115]:
ds_joined.head()

,Request Identifier,Activity ID,Task Executer,Task executer department,Task Type,Action,idBPMApplicationAction,Target,Sex,BirthYear,Role ID,Is Manager,OrgUnitSince,IsOutSourcer,idField,Value,idBPMRequirement,task_ended_before_predicted
0,1,100,7638,569,Initial Request,Submit initial request,270,Request Finished,M,1960-1969,5,Yes,2021,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",Not Applicable,Unknown
1,1,102,4322,1104,Execution,Unknown,273,Request Finished,F,1960-1969,21,No,2024,Y,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",Not Applicable,Yes
2,1,103,10606,Unknown,Execution,Unknown,273,Request Finished,F,1970-1979,11,Yes,2023,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",Not Applicable,Yes
3,1,104,7638,569,Execution,Task executed with success,282,Request Finished,M,1960-1969,5,Yes,2021,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",Not Applicable,No
4,1,107,7638,569,Final task,Request accepted by requester,other,Request Finished,M,1960-1969,5,Yes,2021,N,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",Not Applicable,Yes


In [116]:
# Check if we can drop 'Is Manager' or 'IsOutSourcer' due to their correlation

# Create a new df that only includes the rows when 'Is Manager' is Yes
is_manager_yes = ds_joined[ds_joined['Is Manager'] == 'Yes']
is_manager_yes['Is Manager'].value_counts(dropna=False)

Yes    131965
Name: Is Manager, dtype: int64

In [117]:
is_manager_yes['IsOutSourcer'].value_counts(dropna=False)

N    131965
Name: IsOutSourcer, dtype: int64

In [118]:
# Create a new df that only includes the rows when 'Is Manager' is No
is_manager_no = ds_joined[ds_joined['Is Manager'] == 'No']
is_manager_no['Is Manager'].value_counts(dropna=False)

No    41238
Name: Is Manager, dtype: int64

In [119]:
is_manager_no['IsOutSourcer'].value_counts(dropna=False)

Y    41238
Name: IsOutSourcer, dtype: int64

In [120]:
# Drop 'IsOutSourcer' column
ds_joined = ds_joined.drop(columns=['IsOutSourcer'])

# 3. Group and aggregate dataframes

In [121]:
# Function to aggregate dataframes
def group_and_aggregate_dataframe(df):        
    # Define the aggregation dictionary
    aggregation_dict = {
        "Activity ID": lambda x: list(x),
        "Task Executer": lambda x: list(x),
        "Task executer department": lambda x: list(x),
        "Task Type": lambda x: list(x),
        "Action": lambda x: list(x),
        "idBPMApplicationAction": lambda x: list(x),
        "Target": 'first',
        "Sex": lambda x: list(x),
        "BirthYear": lambda x: list(x),
        "Role ID": lambda x: list(x),
        'Is Manager': lambda x: list(x),
        "OrgUnitSince": lambda x: list(x),
        "idField": 'first',
        "Value": 'first',
        "idBPMRequirement": lambda x: list(x),
        "task_ended_before_predicted": lambda x: list(x),
    }
    
    # Group by 'Request Identifier' and perform aggregation
    grouped_df = df.groupby("Request Identifier").agg(aggregation_dict).reset_index()
    
    return grouped_df

In [122]:
# Apply the function to our dataframe
ds_grouped = group_and_aggregate_dataframe(ds_joined)

In [123]:
ds_grouped.head()

,Request Identifier,Activity ID,Task Executer,Task executer department,Task Type,Action,idBPMApplicationAction,Target,Sex,BirthYear,Role ID,Is Manager,OrgUnitSince,idField,Value,idBPMRequirement,task_ended_before_predicted
0,1,"[100, 102, 103, 104, 107]","[7638, 4322, 10606, 7638, 7638]","[569, 1104, Unknown, 569, 569]","[Initial Request, Execution, Execution, Execut...","[Submit initial request, Unknown, Unknown, Tas...","[270, 273, 273, 282, other]",Request Finished,"[M, F, F, M, M]","[1960-1969, 1960-1969, 1970-1979, 1960-1969, 1...","[5, 21, 11, 5, 5]","[Yes, No, Yes, Yes, Yes]","[2021, 2024, 2023, 2021, 2021]","1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90","[Not Applicable, Not Applicable, Not Applicabl...","[Unknown, Yes, Yes, No, Yes]"
1,2,"[100, 102, 103, 104, 107]","[598, 4322, 10606, 598, 598]","[606, 1104, Unknown, 606, 606]","[Initial Request, Execution, Execution, Execut...","[Submit initial request, Unknown, Unknown, Tas...","[270, 273, 273, 282, other]",Request Finished,"[F, F, F, F, F]","[1970-1979, 1960-1969, 1970-1979, 1970-1979, 1...","[5, 21, 11, 5, 5]","[Yes, No, Yes, Yes, Yes]","[2023, 2024, 2023, 2023, 2023]","3486,3491,3388,3390,3391,3420,203","10,10,10,10,10,20,Unknown","[Not Applicable, Not Applicable, Not Applicabl...","[Unknown, Yes, Yes, Yes, Yes]"
2,3,"[100, 102]","[11029, 11029]","[167, 167]","[Initial Request, Execution]","[Submit initial request, Unknown]","[270, other]",Request Canceled,"[F, F]","[1970-1979, 1970-1979]","[5, 5]","[Yes, Yes]","[2021, 2021]","3390,3420,203,3388,3486","10,10,Unknown,30,30","[Not Applicable, Not Applicable]","[Unknown, Yes]"
3,4,"[100, 102, 102, 101]","[924, 538, 507, Unknown]","[569, 1301, 1336, Unknown]","[Initial Request, Execution, Execution, Reques...","[Submit initial request, Task returned to team...","[270, other, other, other]",Request Canceled,"[M, M, M, Unknown]","[1980-1989, 1980-1989, 1980-1989, Unknown]","[6, other, 11, Unknown]","[Yes, Yes, Yes, Unknown]","[2020, other, other, Unknown]","3486,3388,3390,3420,203","10,10,10,20,Unknown","[Not Applicable, Not Applicable, 191, Not Appl...","[Unknown, No, No, No]"
4,5,"[100, 102, 102, 103, 104]","[8601, 507, 4322, 73, 8601]","[167, 1336, 1104, Unknown, 167]","[Initial Request, Execution, Execution, Execut...","[Submit initial request, Task returned to team...","[270, other, 273, 273, 8888]",Closed administratively Requester Rejects Acco...,"[M, M, F, M, M]","[1960-1969, 1980-1989, 1960-1969, 1970-1979, 1...","[5, 11, 21, 8, 5]","[Yes, Yes, No, Yes, Yes]","[2021, other, 2024, 2022, 2021]","3391,3420,3491,3388,203,3390,3486","10,10,20,20,Unknown,30,30","[Not Applicable, Not Applicable, Not Applicabl...","[Unknown, No, No, No, No]"


# 4. Create new prefix dataframes

In [124]:
# Function to create prefix dataframes with n activities

def create_prefix_dfs(df, max_values=7):
    
    subset_dfs = []    
    
    for n in range(1, max_values + 1):
        subset_df = df.applymap(lambda x: x[:n] if isinstance(x, list) and len(x) >= n else x if not isinstance(x, list) else None)
        subset_df.dropna(axis=1, how='all', inplace=True)
        # Drop rows with any 'None' values
        subset_df.dropna(axis=0, how='any', inplace=True)
        subset_dfs.append(subset_df)
    
    return subset_dfs

In [125]:
# Apply function to our dataframe
subset_dfs = create_prefix_dfs(ds_grouped)

In [126]:
# Naming the prefix dataframes
prefix1, prefix2, prefix3, prefix4, prefix5, prefix6, prefix7 = subset_dfs

In [127]:
prefix3

,Request Identifier,Activity ID,Task Executer,Task executer department,Task Type,Action,idBPMApplicationAction,Target,Sex,BirthYear,Role ID,Is Manager,OrgUnitSince,idField,Value,idBPMRequirement,task_ended_before_predicted
0,1,"[100, 102, 103]","[7638, 4322, 10606]","[569, 1104, Unknown]","[Initial Request, Execution, Execution]","[Submit initial request, Unknown, Unknown]","[270, 273, 273]",Request Finished,"[M, F, F]","[1960-1969, 1960-1969, 1970-1979]","[5, 21, 11]","[Yes, No, Yes]","[2021, 2024, 2023]","1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90","[Not Applicable, Not Applicable, Not Applicable]","[Unknown, Yes, Yes]"
1,2,"[100, 102, 103]","[598, 4322, 10606]","[606, 1104, Unknown]","[Initial Request, Execution, Execution]","[Submit initial request, Unknown, Unknown]","[270, 273, 273]",Request Finished,"[F, F, F]","[1970-1979, 1960-1969, 1970-1979]","[5, 21, 11]","[Yes, No, Yes]","[2023, 2024, 2023]","3486,3491,3388,3390,3391,3420,203","10,10,10,10,10,20,Unknown","[Not Applicable, Not Applicable, Not Applicable]","[Unknown, Yes, Yes]"
3,4,"[100, 102, 102]","[924, 538, 507]","[569, 1301, 1336]","[Initial Request, Execution, Execution]","[Submit initial request, Task returned to team...","[270, other, other]",Request Canceled,"[M, M, M]","[1980-1989, 1980-1989, 1980-1989]","[6, other, 11]","[Yes, Yes, Yes]","[2020, other, other]","3486,3388,3390,3420,203","10,10,10,20,Unknown","[Not Applicable, Not Applicable, 191]","[Unknown, No, No]"
4,5,"[100, 102, 102]","[8601, 507, 4322]","[167, 1336, 1104]","[Initial Request, Execution, Execution]","[Submit initial request, Task returned to team...","[270, other, 273]",Closed administratively Requester Rejects Acco...,"[M, M, F]","[1960-1969, 1980-1989, 1960-1969]","[5, 11, 21]","[Yes, Yes, No]","[2021, other, 2024]","3391,3420,3491,3388,203,3390,3486","10,10,20,20,Unknown,30,30","[Not Applicable, Not Applicable, Not Applicable]","[Unknown, No, No]"
5,6,"[100, 102, 102]","[11029, 507, 4322]","[167, 1336, 1104]","[Initial Request, Execution, Execution]","[Submit initial request, Task returned to team...","[270, other, 273]",Closed administratively Requester Rejects Acco...,"[F, M, F]","[1970-1979, 1980-1989, 1960-1969]","[5, 11, 21]","[Yes, Yes, No]","[2021, other, 2024]","3391,3420,3491,3388,203,3390,3486","10,10,20,20,Unknown,30,30","[Not Applicable, Not Applicable, Not Applicable]","[Unknown, No, No]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43943,45768,"[100, 102, 102]","[61, 6266, 10606]","[453, 1146, 1147]","[Initial Request, Execution, Execution]","[Submit initial request, Task returned to team...","[270, other, 273]",Request Finished,"[F, M, F]","[1970-1979, 1990-1999, 1970-1979]","[5, 21, 11]","[Yes, No, Yes]","[2023, 2023, 2023]","3390,3491,203,3388,3486,3391","10,10,Unknown,50,50,70","[Not Applicable, Not Applicable, Not Applicable]","[Unknown, Yes, No]"
43944,45769,"[100, 102, 103]","[9549, 6266, 10606]","[347, 1146, 1147]","[Initial Request, Execution, Execution]","[Submit initial request, Unknown, Unknown]","[270, 273, 300]",Request Finished,"[M, M, F]","[1970-1979, 1990-1999, 1970-1979]","[5, 21, 11]","[Yes, No, Yes]","[2024, 2023, 2023]","3491,3390,203,3388,3486,3391","10,100,Unknown,50,50,70","[Not Applicable, Not Applicable, Not Applicable]","[Unknown, Yes, Yes]"
43945,45770,"[100, 102, 102]","[7351, 6266, 10606]","[585, 1146, 1147]","[Initial Request, Execution, Execution]","[Submit initial request, Task returned to team...","[270, other, 275]",Request Finished,"[M, M, F]","[1960-1969, 1990-1999, 1970-1979]","[4, 21, 11]","[Yes, No, Yes]","[2022, 2023, 2023]","3390,1604,47,1602,203,3486,3388,3935,3391","10,100,2,2,Unknown,30,40,50,60","[Not Applicable, Not Applicable, Not Applicable]","[Unknown, Yes, Yes]"
43946,45771,"[100, 102, 104]","[7945, 8548, 7945]","[478, 1147, 478]","[Initial Request, Execution, Execution]","[Submit initial request, Unknown, Task execute...","[270, 275, 282]",Request Finished,"[M, 

In [128]:
# Transforming lists to different columns 

In [129]:
exclude_columns = ['Request Identifier', 'Target', 'idField', 'Value']

# Function to expand columns
def expand_columns(df, exclude_columns):
    expanded_df = df[exclude_columns].copy()
    
    for column in df.columns:
        if column not in exclude_columns:
            # Generate new column names with the prefix
            num_columns = len(df[column].iloc[0])
            column_names = [f'{i+1}_{column}' for i in range(num_columns)]
            # Expand the current column into separate columns with custom names
            values_df = pd.DataFrame(df[column].tolist(), index=df.index, columns=column_names)
            # Add the expanded columns to the expanded_df
            expanded_df = expanded_df.join(values_df)
    
    return expanded_df

# Apply function to the prefix dataframes
expanded_dfs = [expand_columns(df, exclude_columns) for df in subset_dfs]

In [130]:
# Naming
prefix1, prefix2, prefix3, prefix4, prefix5, prefix6, prefix7 = expanded_dfs

In [131]:
prefix1

,Request Identifier,Target,idField,Value,1_Activity ID,1_Task Executer,1_Task executer department,1_Task Type,1_Action,1_idBPMApplicationAction,1_Sex,1_BirthYear,1_Role ID,1_Is Manager,1_OrgUnitSince,1_idBPMRequirement,1_task_ended_before_predicted
0,1,Request Finished,"1602,47,3486,3491,3388,3391,3420,203,1604","1,1,10,10,10,10,20,Unknown,90",100,7638,569,Initial Request,Submit initial request,270,M,1960-1969,5,Yes,2021,Not Applicable,Unknown
1,2,Request Finished,"3486,3491,3388,3390,3391,3420,203","10,10,10,10,10,20,Unknown",100,598,606,Initial Request,Submit initial request,270,F,1970-1979,5,Yes,2023,Not Applicable,Unknown
2,3,Request Canceled,"3390,3420,203,3388,3486","10,10,Unknown,30,30",100,11029,167,Initial Request,Submit initial request,270,F,1970-1979,5,Yes,2021,Not Applicable,Unknown
3,4,Request Canceled,"3486,3388,3390,3420,203","10,10,10,20,Unknown",100,924,569,Initial Request,Submit initial request,270,M,1980-1989,6,Yes,2020,Not Applicable,Unknown
4,5,Closed administratively Requester Rejects Acco...,"3391,3420,3491,3388,203,3390,3486","10,10,20,20,Unknown,30,30",100,8601,167,Initial Request,Submit initial request,270,M,1960-1969,5,Yes,2021,Not Applicable,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43943,45768,Request Finished,"3390,3491,203,3388,3486,3391","10,10,Unknown,50,50,70",100,61,453,Initial Request,Submit initial request,270,F,1970-1979,5,Yes,2023,Not Applicable,Unknown
43944,45769,Request Finished,"3491,3390,203,3388,3486,3391","10,100,Unknown,50,50,70",100,9549,347,Initial Request,Submit initial request,270,M,1970-1979,5,Yes,2024,Not Applicable,Unknown
43945,45770,Request Finished,"3390,1604,47,1602,203,3486,3388,3935,3391","10,100,2,2,Unknown,30,40,50,60",100,7351,585,Initial Request,Submit initial request,270,M,1960-1969,4,Yes,2022,Not Applicable,Unknown
43946,45771,Request Finished,"47,1602,3390,203,3388,3486,3935,3391,1604","1,1,20,Unknown,30,30,50,60,90",100,7945,478,Initial Request,Submit initial request,270,M,1960-1969,4,Yes,2023,Not Applicable,Unknown


# 5. Encoding 

In [132]:
# Encoding methods for each variable
config = {
    'Task Executer': 'frequency',
    'Task executer department': 'frequency',
    'Activity ID': 'one_hot',
    'Task Type': 'one_hot',
    'Action': 'one_hot',
    'idBPMApplicationAction': 'one_hot',
    'Sex': 'label',
    'BirthYear': 'one_hot',
    'Role ID': 'one_hot',
    'Is Manager': 'label',
    'OrgUnitSince': 'one_hot',
    'idField': 'frequency',
    'Value': 'frequency',
    'idBPMRequirement': 'binary',
    'task_ended_before_predicted': 'binary'
}

In [133]:
# Frequency Encoding Function
def frequency_encoding(df, column):
    freq_encoding = df[column].value_counts(normalize=True)
    df[column] = df[column].map(freq_encoding)
    return df

# Binary Encoding Function
def binary_encoding(df, column):
    df[column] = df[column].map(lambda x: 0 if x == 'Not Applicable' else 1)
    return df

# Function to apply encodings based on the config and prefixed column names
def apply_encodings(df, config):
    df_encoded = df.copy()
    transformers = defaultdict(list)
    
    if 'idField' in df_encoded.columns:
        df_encoded = frequency_encoding(df_encoded, 'idField')
    if 'Value' in df_encoded.columns:
        df_encoded = frequency_encoding(df_encoded, 'Value')
        
    # Populate transformers based on the config, excluding 'idField' and 'Value'
    for col_base, encoding_type in config.items():
        if col_base in ['idField', 'Value']:
            continue
        max_col_count = max(
            int(col.split('_')[0]) for col in df_encoded.columns if col.endswith(col_base)
        ) + 1
        for i in range(1, max_col_count):
            col = f'{i}_{col_base}'
            if col not in df_encoded.columns:
                print(f"Column {col} does not exist for encoding type {encoding_type}.")
                continue
            if encoding_type == 'one_hot':
                transformers['one_hot'].append(col)
            elif encoding_type == 'label':
                transformers['label'].append(col)
            elif encoding_type == 'frequency':
                df_encoded = frequency_encoding(df_encoded, col)
            elif encoding_type == 'binary':
                df_encoded = binary_encoding(df_encoded, col)
                
    # Apply One-Hot and Label Encodings
    for encoding_type, columns in transformers.items():
        if encoding_type == 'one_hot':
            for column in columns:
                ohc = OneHotEncoder(sparse_output=False, drop='first')
                ohc_feat = ohc.fit_transform(df_encoded[[column]])
                ohc_feat_names = ohc.get_feature_names_out([column])
                encoded_df = pd.DataFrame(ohc_feat, index=df_encoded.index, columns=ohc_feat_names)
                df_encoded = pd.concat([df_encoded.drop(columns=column), encoded_df], axis=1)
        elif encoding_type == 'label':
            for column in columns:
                mapping = {val: idx for idx, val in enumerate(df_encoded[column].unique())}
                df_encoded[column] = df_encoded[column].map(mapping)
    return df_encoded

In [134]:
# Apply function to our dataframes

prefix1 = apply_encodings(prefix1, config)
prefix2 = apply_encodings(prefix2, config)
prefix3 = apply_encodings(prefix3, config)
prefix4 = apply_encodings(prefix4, config)
prefix5 = apply_encodings(prefix5, config)
prefix6 = apply_encodings(prefix6, config)
prefix7 = apply_encodings(prefix7, config)

In [135]:
prefix2

,Request Identifier,Target,idField,Value,1_Task Executer,2_Task Executer,1_Task executer department,2_Task executer department,1_Sex,2_Sex,...,1_OrgUnitSince_2024,1_OrgUnitSince_Unknown,1_OrgUnitSince_other,2_OrgUnitSince_2020,2_OrgUnitSince_2021,2_OrgUnitSince_2022,2_OrgUnitSince_2023,2_OrgUnitSince_2024,2_OrgUnitSince_Unknown,2_OrgUnitSince_other
0,1,Request Finished,0.000091,0.000091,0.005415,0.199577,0.008123,0.445891,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2,Request Finished,0.002162,0.002321,0.002526,0.199577,0.011241,0.445891,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,Request Canceled,0.000023,0.000137,0.000978,0.000023,0.005825,0.000023,1,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4,Request Canceled,0.000023,0.000114,0.000569,0.000023,0.008123,0.000023,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,Closed administratively Requester Rejects Acco...,0.000228,0.000455,0.002071,0.008396,0.005825,0.011991,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43943,45768,Request Finished,0.011787,0.000796,0.003436,0.016497,0.009352,0.438131,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
43944,45769,Request Finished,0.006280,0.002275,0.000705,0.016497,0.006872,0.438131,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
43945,45770,Request Finished,0.000114,0.000387,0.004938,0.016497,0.007805,0.438131,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
43946,45771,Request Finished,0.000137,0.000023,0.002526,0.031674,0.000114,0.082666,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# 6. Modelling and predictions scores  

In [136]:
dataframes = [prefix1, prefix2, prefix3, prefix4, prefix5, prefix6, prefix7]

In [137]:
# Function to execute gridsearch 

def execute_gridSearch_allModels(df, X, y, *model_param_pairs):
    best_models = {}
    for idx, (model_instance, param_grid) in enumerate(model_param_pairs):
        cv_num = 5
        grid = GridSearchCV(estimator=model_instance, param_grid=param_grid, cv=cv_num, scoring='f1_macro', n_jobs=-1, return_train_score=True) #  verbose=3,
        start_time = time.perf_counter()
        grid.fit(X, y)
        end_time = time.perf_counter()

        best_params = grid.best_params_
        best_estimator = grid.best_estimator_

        # Store the best model
        model_name = df.index[idx]
        best_models[model_name] = best_estimator

        # Extracting performance metrics
        avg_time = (end_time - start_time) / cv_num
        avg_f1_train = np.mean(grid.cv_results_['mean_train_score'])
        avg_f1_test = np.mean(grid.cv_results_['mean_test_score'])
        avg_acc_train = np.mean(cross_val_score(best_estimator, X, y, cv=cv_num, scoring='accuracy', n_jobs=-1))
        avg_acc_test = np.mean(cross_val_score(best_estimator, X, y, cv=cv_num, scoring='accuracy', n_jobs=-1))
        avg_rec_train = np.mean(cross_val_score(best_estimator, X, y, cv=cv_num, scoring='recall_macro', n_jobs=-1))
        avg_rec_test = np.mean(cross_val_score(best_estimator, X, y, cv=cv_num, scoring='recall_macro', n_jobs=-1))

        # Update the DataFrame
        df.iloc[idx] = [avg_time, avg_f1_train, avg_f1_test, avg_acc_train, avg_acc_test, avg_rec_train, avg_rec_test]

    return df, best_models

In [138]:
# Model parameters

# Logistic Regression
param_grid_log = {
    'C': [0.0001, 0.001, 0.005, 0.01, 0.1],  
    'solver': ['lbfgs', 'saga'],
    'max_iter': [100, 300, 500],
    'class_weight': ['balanced']  
}

# Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 150, 200],
    # 'criterion': ["gini", "entropy"],
    'max_depth': [5, 7, 10, 13, 20], 
    'class_weight': ['balanced', 'balanced_subsample'] 
}

# Decision Tree
param_grid_dt = {
    'max_depth': [5, 7, 10, 13, 20],  
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced']  
}

In [154]:
# Function to split data, select features with RFE

def preprocess_data(df):
    df = df.drop('Request Identifier', axis=1)
    
    X = df.drop(['Target'], axis=1)
    y = df['Target']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    X_train_numerical = X_train.select_dtypes(include=np.number).set_index(X_train.index)
    X_train_categorical = X_train.select_dtypes(exclude=np.number).set_index(X_train.index)
    X_test_numerical = X_test.select_dtypes(include=np.number).set_index(X_test.index)
    X_test_categorical = X_test.select_dtypes(exclude=np.number).set_index(X_test.index)
    
    scaler = MinMaxScaler()
    scaler.fit(X_train_numerical)
    X_train_num_scaled = scaler.transform(X_train_numerical)
    scaler.fit(X_test_numerical)
    X_test_num_scaled = scaler.transform(X_test_numerical)
    
    # Convert the array to a pandas dataframe
    X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns = X_train_numerical.columns).set_index(X_train.index)
    X_test_num_scaled = pd.DataFrame(X_test_num_scaled, columns = X_test_numerical.columns).set_index(X_test.index)
    
    # Split the training data
    X_train_num_scaled_rfe, X_val_num_scaled_rfe, y_train_rfe, y_val_rfe = train_test_split(X_train_num_scaled, y_train, test_size=0.2, random_state=42)
    
    # RFE with base estimator Logistic Regression
    model = LogisticRegression(max_iter=100)
    rfe = RFE(estimator=model, n_features_to_select=30)
    X_rfe = rfe.fit_transform(X=X_train_num_scaled, y=y_train)
    
    # Selected features
    selected_features_rfe = pd.Series(rfe.support_, index=X_train_num_scaled.columns)
    
    # Filter the dataframes to include only selected features
    selected_columns = selected_features_rfe[selected_features_rfe].index.tolist()
    X_train_num_scaled = X_train_num_scaled[selected_columns]
    X_test_num_scaled = X_test_num_scaled[selected_columns]
    
    # Encoding our target variable
    y_encoded = y_train.map({
        'Request Finished': 3, 
        'Closed administratively Requester Rejects Accounting Impact': 2, 
        'Request Canceled': 1, 
        'Closed administratively': 0
    })
    
    # Basic Renaming
    X_train = X_train_num_scaled
    y_train = y_encoded
    
    return X_train, y_train

In [155]:
# Apply the preprocessing function to each prefix
processed_dataframes = [preprocess_data(df) for df in dataframes]

In [156]:
# Unpacking the results into separate variables for each prefix
X_train_prefix1, y_train_prefix1 = processed_dataframes[0]
X_train_prefix2, y_train_prefix2 = processed_dataframes[1]
X_train_prefix3, y_train_prefix3 = processed_dataframes[2]
X_train_prefix4, y_train_prefix4 = processed_dataframes[3]
X_train_prefix5, y_train_prefix5 = processed_dataframes[4]
X_train_prefix6, y_train_prefix6 = processed_dataframes[5]
X_train_prefix7, y_train_prefix7 = processed_dataframes[6]

In [161]:
X_train_prefix7

,6_Task Executer,6_Is Manager,5_idBPMRequirement,6_idBPMRequirement,6_Activity ID_102,6_Activity ID_103,6_Activity ID_104,6_Activity ID_107,7_Activity ID_104,7_Activity ID_107,...,5_idBPMApplicationAction_other,6_idBPMApplicationAction_282,6_idBPMApplicationAction_300,6_idBPMApplicationAction_other,7_idBPMApplicationAction_8888,7_idBPMApplicationAction_other,7_BirthYear_1970-1979,7_BirthYear_Unknown,6_Role ID_21,7_Role ID_Unknown
23330,0.032787,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
28581,1.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
16081,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
19726,0.043716,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
29981,0.005464,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40292,0.005464,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
40714,0.005464,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
42762,0.021858,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36455,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [162]:
# Prefix 1 grid search and modelling
df = pd.DataFrame(columns=['Time', 'avg_f1_train', 'avg_f1_test', "avg_acc_train", "avg_acc_test", "avg_rec_train", "avg_rec_test"], 
                   index=['LogReg', 'RF', 'DT'])


results_prefix1, best_models_prefix1 = execute_gridSearch_allModels(df, X_train_prefix1, y_train_prefix1, 
                                       (LogisticRegression(), param_grid_log),   
                                       (RandomForestClassifier(), param_grid_rf), 
                                       (DecisionTreeClassifier(), param_grid_dt),
                                       )

In [ ]:
# Prefix 2 grid search and modelling
df = pd.DataFrame(columns=['Time', 'avg_f1_train', 'avg_f1_test', "avg_acc_train", "avg_acc_test", "avg_rec_train", "avg_rec_test"], 
                   index=['LogReg', 'RF', 'DT'])


results_prefix2, best_models_prefix2 = execute_gridSearch_allModels(df, X_train_prefix2, y_train_prefix2, 
                                       (LogisticRegression(), param_grid_log),   
                                       (RandomForestClassifier(), param_grid_rf), 
                                       (DecisionTreeClassifier(), param_grid_dt),
                                       )

In [ ]:
# Prefix 3 grid search and modelling
df = pd.DataFrame(columns=['Time', 'avg_f1_train', 'avg_f1_test', "avg_acc_train", "avg_acc_test", "avg_rec_train", "avg_rec_test"], 
                   index=['LogReg', 'RF', 'DT'])


results_prefix3, best_models_prefix3 = execute_gridSearch_allModels(df, X_train_prefix3, y_train_prefix3, 
                                       (LogisticRegression(), param_grid_log),   
                                       (RandomForestClassifier(), param_grid_rf), 
                                       (DecisionTreeClassifier(), param_grid_dt),
                                       )

In [ ]:
# Prefix 4 grid search and modelling
df = pd.DataFrame(columns=['Time', 'avg_f1_train', 'avg_f1_test', "avg_acc_train", "avg_acc_test", "avg_rec_train", "avg_rec_test"], 
                   index=['LogReg', 'RF', 'DT'])


results_prefix4, best_models_prefix4 = execute_gridSearch_allModels(df, X_train_prefix4, y_train_prefix4, 
                                       (LogisticRegression(), param_grid_log),   
                                       (RandomForestClassifier(), param_grid_rf), 
                                       (DecisionTreeClassifier(), param_grid_dt),
                                       )

In [ ]:
# Prefix 5 grid search and modelling
df = pd.DataFrame(columns=['Time', 'avg_f1_train', 'avg_f1_test', "avg_acc_train", "avg_acc_test", "avg_rec_train", "avg_rec_test"], 
                   index=['LogReg', 'RF', 'DT'])


results_prefix5, best_models_prefix5 = execute_gridSearch_allModels(df, X_train_prefix5, y_train_prefix5, 
                                       (LogisticRegression(), param_grid_log),   
                                       (RandomForestClassifier(), param_grid_rf), 
                                       (DecisionTreeClassifier(), param_grid_dt),
                                       )

In [ ]:
# Prefix 6 grid search and modelling
df = pd.DataFrame(columns=['Time', 'avg_f1_train', 'avg_f1_test', "avg_acc_train", "avg_acc_test", "avg_rec_train", "avg_rec_test"], 
                   index=['LogReg', 'RF', 'DT'])


results_prefix6, best_models_prefix6 = execute_gridSearch_allModels(df, X_train_prefix6, y_train_prefix6, 
                                       (LogisticRegression(), param_grid_log),   
                                       (RandomForestClassifier(), param_grid_rf), 
                                       (DecisionTreeClassifier(), param_grid_dt),
                                       )

In [ ]:
# Prefix 7 grid search and modelling
df = pd.DataFrame(columns=['Time', 'avg_f1_train', 'avg_f1_test', "avg_acc_train", "avg_acc_test", "avg_rec_train", "avg_rec_test"], 
                   index=['LogReg', 'RF', 'DT'])


results_prefix7, best_models_prefix7 = execute_gridSearch_allModels(df, X_train_prefix7, y_train_prefix7, 
                                       (LogisticRegression(), param_grid_log),   
                                       (RandomForestClassifier(), param_grid_rf), 
                                       (DecisionTreeClassifier(), param_grid_dt),
                                       )

In [ ]:
# Prefix 1 predictions results
results_prefix1

In [ ]:
# Prefix 1 best models
best_models_prefix1

In [ ]:
# Prefix 2 predictions results
results_prefix2

In [ ]:
# Prefix 2 best models
best_models_prefix2

In [ ]:
# Prefix 3 predictions results
results_prefix3

In [ ]:
# Prefix 3 best models
best_models_prefix3

In [ ]:
# Prefix 4 predictions results
results_prefix4

In [ ]:
# Prefix 4 best models
best_models_prefix4

In [ ]:
# Prefix 5 predictions results
results_prefix5

In [ ]:
# Prefix 5 best models
best_models_prefix5

In [ ]:
# Prefix 6 predictions results
results_prefix6

In [ ]:
# Prefix 6 best models
best_models_prefix6

In [ ]:
# Prefix 7 predictions results
results_prefix7

In [ ]:
# Prefix 7 best models
best_models_prefix7

## nope

In [ ]:
## Function to plot F1 scores from the results dataframe

#def plot_f1_scores(results_df):
#    sorted_df = results_df.sort_values(by='avg_f1_test', ascending=False)
#
#    # Colors (PANTONE 390 C and PANTONE 431 C)
#    color_train = (190/255, 214/255, 47/255)  # Green
#    color_test = (92/255, 102/255, 108/255)  # Grey
#
#    # Extracting model names and F1 scores
#    models = sorted_df.index.tolist()
#    avg_f1_train = sorted_df['avg_f1_train'].tolist()
#    avg_f1_test = sorted_df['avg_f1_test'].tolist()
#
#    # Setting up the plot
#    plt.figure(figsize=(8, 5))
#    bar_width = 0.35
#    index = np.arange(len(models))
#
#    # Plotting the bars
#    plt.bar(index - bar_width/2, avg_f1_train, bar_width, color=color_train, label='Train')
#    plt.bar(index + bar_width/2, avg_f1_test, bar_width, color=color_test, label='Test')
#
#    plt.xlabel('Models', labelpad=10)
#    plt.ylabel('Average F1 Score', labelpad=10)
#    plt.xticks(index, models, rotation=45)  # Rotate model names for better readability
#    plt.legend()
#    plt.tight_layout()
#    plt.show()

In [ ]:
# plot_f1_scores(results_df)

In [ ]:
##no of features
#nof_list=np.arange(1,35)            
#high_score=0
##Variable to store the optimum features
#nof=0           
#train_score_list =[]
#val_score_list = []
#
#for n in range(len(nof_list)):
#    model = LogisticRegression(max_iter=1000)
#    
#    rfe = RFE(estimator = model,n_features_to_select = nof_list[n])
#    X_train_rfe = rfe.fit_transform(X_train_num_scaled_rfe,y_train_rfe)
#    X_val_rfe = rfe.transform(X_val_num_scaled_rfe)
#    model.fit(X_train_rfe,y_train_rfe)
#    
#    #storing results on training data
#    train_score = model.score(X_train_rfe,y_train_rfe)
#    train_score_list.append(train_score)
#    
#    #storing results on validation data
#    val_score = model.score(X_val_rfe,y_val_rfe)
#    val_score_list.append(val_score)
#    
#    #check best score
#    if(val_score >= high_score):
#        high_score = val_score
#        nof = nof_list[n]
#        
#print("Optimum number of features: %d" %nof)
#print("Score with %d features: %f" % (nof, high_score))

In [ ]:
# https://machinelearningtutorials.org/pandas-encoding-categorical-features-with-examples/

In [ ]:
# Data Visualization (Target proportion)
ds_joined.Target.value_counts().plot.pie(autopct = "%.1f%%")
plt.title("Proportion")
plt.show()